# Medindo a adaptabilidade dos alunos à educação online

---

# 1. Problema de Negócio

A pandemia trouxe diversas mudanças no nosso dia-a-dia, e a mais impactante (além do problema sanitário) foi a transferência do ensino presencial para o ensino à distância, que impactou diretamente a vida de milhões de estudantes ao redor do mundo. Neste projeto, utilizaremos modelos de classificação para saber qual o grau de adaptabilidade dos estudantes e o que pode ter influenciado essa medida.

# 2. Modelo

O modelo utilizado neste relatório é o K-Nearest Neighbors(KNN), um algoritmo de aprendizagem supervisionada que busca mostrar a similaridade entre vetores(ou dados) por meio da medição da distância entre eles. Quanto mais próximos os 'pontos', mais semelhantes são os dados. O algoritmo KNN utiliza de várias métricas que medem distância entre pontos, como por exemplo, a distância euclidiana, que usa o Teorema de Pitágoras. Com isso, o programador escolhe quantos pontos(ou vizinhos) próximos ele quer considerar no modelo. A imagem a seguir ilustra isso:

<img src="https://www.jcchouinard.com/wp-content/uploads/2021/08/image-8.png.webp" width="35%" />

# 3. Preparação dos Dados

### Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pylab import rcParams
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,fbeta_score, precision_score, recall_score

from sklearn import metrics, preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

### Tratamento dos dados

In [2]:
df = pd.read_csv('students_adaptability_level_online_education.csv')
df.pop('Class Duration')
df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'students_adaptability_level_online_education.csv'

- Removemos a coluna "Class Duration" porque a métrica utilizada nessa coluna é incerta

---

In [ ]:
df.isna().any()

- O dataset não possui valores *NaN*

---

In [ ]:
le = LabelEncoder()
df_fact = df.apply(le.fit_transform)
df_fact.head(5)

In [ ]:
df_fact.dtypes

- Aqui estamos 'rotulando' o dataset. O modelo KNN não consegue trabalhar com strings(palavras), portanto precisamos transformá-las em números para se tornarem 'legíveis' para a máquina.

---

In [ ]:
# Normalizando todos os valores de 'df_fact', exceto a coluna 'Adaptivity Level', que é o Target
scaler = StandardScaler()
df_norm = scaler.fit_transform(df_fact.loc[:, df_fact.columns != "Adaptivity Level"])
df_norm

- Normalizar as features do dataset costuma ajudar no treinamento e geralmente aumenta um pouco a precisão do modelo

---

# Treinamento do Modelo

In [ ]:
# Separando as features e o target
X = df_norm
y = df_fact['Adaptivity Level']

In [ ]:
# Separando os dados de treino e de teste (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 109)

- Separamos as **features** ( "características" de cada estudante) e o **target**( "resultado" da classificação) em **X** e **Y**, respectivamente.   
      
- Utilizamos uma função do ScikitLearn chamada **TrainTestSplit**, que separa os dados em dados de treino do modelo e em dados para testar o modelo. A proporção é de **70/30%**, respectivamente. O "**random_state**" serve para manter os dados de treinamento e teste iguais em todas as vezes que o comando for executado, caso contrário, os dados de treino e teste seriam aleatorizados e a precisão do modelo oscilaria.

---

### Métricas de Desempenho

In [ ]:
precisao = {}

def metricas(k, metrica):
    for i in range(1, 21):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train,y_train)
        y_pred = knn.predict(X_test)
        if metrica == accuracy_score:
            precisao.update({k:round(metrica(y_test, y_pred), 3)})
        elif metrica == fbeta_score:
            precisao.update({k:round(metrica(y_test, y_pred, average = 'macro', beta=1.0), 3)})
        else:
            precisao.update({k:round(metrica(y_test, y_pred, average = 'macro'), 3)})
        k += 1
    print('k =',max(precisao, key=precisao.get),'/', max(precisao.values()))

- Aqui utilizamos uma função para testar a variável K com todos os números de 1 a 15, para assim sabermos qual o valor de K trará a maior precisão

---

- **Precisão**

In [ ]:
metricas(1, precision_score)

- **Acurácia**

In [ ]:
metricas(1, accuracy_score)

- **Recall**

In [ ]:
metricas(1, recall_score)

- **F-Beta**

In [ ]:
metricas(1, fbeta_score)

---

**Obs: Para validação cruzada e para a matriz de confusão, o valor de K utilizado é 3**

- **Validação Cruzada**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

In [ ]:
k_folds = KFold(n_splits = 5)

scores = cross_val_score(knn, X, y, cv = k_folds)

print(scores)

- **Matriz de Confusão**

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['High','Low','Moderate'])

cm_display.plot()
plt.show() 

---

# Visualização dos Dados

- **Matriz de Correlação entre as Variáveis**

In [ ]:
def matriz_de_confusão(data):
    # init figure size
    rcParams['figure.figsize'] = 10, 10
    fig = plt.figure()
    sns.heatmap(data.corr(), annot=True, fmt=".2f")
    plt.show()
    #fig.savefig('corr.png')

matriz_de_confusão(df_fact)

# Conclusão

O KNN é considerado um modelo simples e fácil de entender, com treinamento rápido e pouca exigência computacional, o que torna o K-Nearest Neighbors recomendado para hardwares limitados. Porém, essa simplicidade sacrifica um pouco da precisão do modelo tornando-o menos atrativo para o uso empresarial/científico.
Nos dias de hoje, o modelo KNN fica em segundo plano, já que o poder computacional não é mais um problema como era antigamente, o que dá mais espaço à modelos mais robustos (e mais precisos).

---